In [6]:
from pathlib import Path
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from longformer.longformer import LongformerConfig, LongformerForSequenceClassification


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = torch.device('cpu')
model_path = Path("../../models/longformer_zh")
model_path = Path("../../models/distilbert-base-uncased")
tokenizer = BertTokenizer.from_pretrained(model_path)
config = LongformerConfig.from_pretrained(model_path)
config = LongformerConfig.from_json_file(model_path /"config.json")
config.problem_type = "multi_label_classification"
# config.problem_type = "single_label_classification"
config.num_labels = 6
# model = LongformerForSequenceClassification.from_pretrained(model_path, config=config)
model = BertForSequenceClassification.from_pretrained(model_path, config=config)
model.to(device)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'DistilBertTokenizer'. 
The class this function is called from is 'BertTokenizer'.
You are using a model of type distilbert to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.
Some weights of the model checkpoint at ../../models/distilbert-base-uncased were not used when initializing BertForSequenceClassification: ['distilbert.transformer.layer.3.ffn.lin1.weight', 'distilbert.transformer.layer.2.output_layer_norm.bias', 'distilbert.transformer.layer.2.attention.q_lin.weight', 'distilbert.transformer.layer.5.attention.v_lin.bias', 'distilbert.transformer.layer.1.output_layer_norm.bias', 'distilbert.transformer.layer.2.ffn.lin2.weight', 'distilbert.transformer.layer.2.attention.k_lin.bias', 'distilbert.transformer.layer.

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [7]:
import pandas as pd 

train_path = "../data/MultilabelSequenceClassification/toxic-comment-classification/train.csv.zip"
test_path = "../data/MultilabelSequenceClassification/toxic-comment-classification/test.csv.zip"
df = pd.read_csv(train_path)
df['label'] = df[df.columns[2:]].values.tolist()
new_df = df[['comment_text', 'label']].copy()
print(new_df.head())

train_size=0.9
test_data = pd.read_csv(test_path)[:2]
train_data = new_df.sample(frac=train_size,random_state=200).reset_index(drop=True)[:4]
val_data = new_df.drop(train_data.index).reset_index(drop=True)[:1]

print("FULL Dataset: {}".format(new_df.shape))
print("TRAIN Dataset: {}".format(train_data.shape))
print("TEST Dataset: {}".format(test_data.shape))

del df
del new_df

                                        comment_text               label
0  Explanation\nWhy the edits made under my usern...  [0, 0, 0, 0, 0, 0]
1  D'aww! He matches this background colour I'm s...  [0, 0, 0, 0, 0, 0]
2  Hey man, I'm really not trying to edit war. It...  [0, 0, 0, 0, 0, 0]
3  "\nMore\nI can't make any real suggestions on ...  [0, 0, 0, 0, 0, 0]
4  You, sir, are my hero. Any chance you remember...  [0, 0, 0, 0, 0, 0]
FULL Dataset: (159571, 2)
TRAIN Dataset: (4, 2)
TEST Dataset: (2, 2)


In [45]:
import torch
from torch.utils.data import Dataset, DataLoader

class CustomDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.text = dataframe['comment_text']
        self.targets = self.data.label
        self.max_len = max_len

    def __len__(self):
        return len(self.text)

    def __getitem__(self, index):
        text = str(self.text[index])
        text = " ".join(text.split())

        inputs = self.tokenizer(
            text,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            return_token_type_ids=True,
            truncation=True,
            return_tensors='pt'
        ).to(device)
        inputs_single = {}
        inputs_single['input_ids'] = inputs['input_ids'][0]
        inputs_single['attention_mask'] = inputs['attention_mask'][0]
        inputs_single['token_type_ids'] = inputs['token_type_ids'][0]
        targets = torch.tensor(self.targets[index], dtype=torch.float)

        return inputs_single, targets

In [46]:
train_datset = CustomDataset(train_data, tokenizer, config.max_position_embeddings)
val_set = CustomDataset(val_data, tokenizer, config.max_position_embeddings)
train_loader = DataLoader(train_datset, batch_size=4, shuffle=True)
val_loader = DataLoader(train_datset, batch_size=4)

In [47]:
for batch, data in enumerate(train_loader):
    print(batch, data)
    inputs, labels = data

0 [{'input_ids': tensor([[ 101, 3231, 4003,  ...,    0,    0,    0],
        [ 101, 2026, 6707,  ...,    0,    0,    0],
        [ 101, 3125, 3195,  ...,    0,    0,    0],
        [ 101, 1000, 1024,  ...,    0,    0,    0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]])}, tensor([[0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.]])]


In [31]:
outputs = model(**inputs)

In [52]:
# Train model 
optimizer = torch.optim.Adam(params =  model.parameters(), lr=1e-05)
loss_fn = torch.nn.BCEWithLogitsLoss()
def train_one_epoch(epoch_index, tb_writer):
    running_loss = 0.
    last_loss = 0.

    # Here, we use enumerate(training_loader) instead of
    # iter(training_loader) so that we can track the batch
    # index and do some intra-epoch reporting
    for i, data in enumerate(train_loader):
        # Every data instance is an input + label pair
        inputs, labels = data

        # Zero your gradients for every batch!
        optimizer.zero_grad()

        # Make predictions for this batch
        outputs = model(**inputs)
        loss = outputs.loss
        # Compute the loss and its gradients
        loss = loss_fn(outputs.logits, labels)
        loss.backward()

        # Adjust learning weights
        optimizer.step()
        print(1)
        # Gather data and report
        running_loss += loss.item()
        if i % 1000 == 999:
            print(1)
            last_loss = running_loss / 1000 # loss per batch
            print('  batch {} loss: {}'.format(i + 1, last_loss))
            tb_x = epoch_index * len(train_loader) + i + 1
            tb_writer.add_scalar('Loss/train', last_loss, tb_x)
            running_loss = 0.

    return last_loss


In [57]:
from datetime import datetime
from torch.utils.tensorboard import SummaryWriter

epochs = 1
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
writer = SummaryWriter('runs/fashion_trainer_{}'.format(timestamp))
best_vloss = 1_000_000.
for epoch in range(1, epochs+1):
    print(f'epoch {epoch}:')  # 输出轮次

    avg_loss = train_one_epoch(epoch, writer)  # 获得一轮结束后平均损失
    
    # model.train(False)
    with torch.no_grad():  # 不求梯度
        running_vloss = 0.0
        for i, (vinputs, vlabels) in enumerate(val_loader):
            voutputs = model(**vinputs)
            vloss = loss_fn(voutputs.logits, vlabels)
            running_vloss += vloss

    avg_vloss = running_vloss / (i + 1)
    print('LOSS train {} valid {}'.format(avg_loss, avg_vloss))
    writer.add_scalars('Training vs. Validation Loss',
                       {'Training': avg_loss, 'Validation': avg_vloss},
                        epoch)  # 写入训练集和测试集损失

    best_vloss
    if avg_loss < best_vloss:  # 如果平均损失小于最小的损失
        best_vloss = avg_loss  # 更新最小损失为平均损失
        model_dirs = f'../../models/longformer_test'
        save_model_path = f'../../models/longformer_test/longformer_test.bin'
        torch.save(model, save_model_path)  # 将模型保存到该路径
        model.save_pretrained(model_dirs)

epoch 1:
1
LOSS train 0.0 valid 0.15499968826770782
